In [221]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor

In [222]:
def recommend_wine(wine_type, grape_variety, gender, occasion):
    # Load the dataset
    df = pd.read_csv('/content/FINDFINEWINE1.csv', encoding='ISO-8859-1')

    # Convert categorical variables to numeric
    le_wine_type = LabelEncoder()
    le_grape_variety = LabelEncoder()
    le_gender = LabelEncoder()
    le_occasion = LabelEncoder()
    le_title = LabelEncoder()

    df['Wine Type'] = le_wine_type.fit_transform(df['Wine Type'])
    df['Grape Variety'] = le_grape_variety.fit_transform(df['Grape Variety'])
    df['Gender'] = le_gender.fit_transform(df['Gender'])
    df['Occasion'] = le_occasion.fit_transform(df['Occasion'])
    df['Title'] = le_title.fit_transform(df['Title'])

    X = df[['Wine Type', 'Grape Variety', 'Gender', 'Occasion']]
    y = df['Title']

    # Fit KNeighborsClassifier model
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(X, y)

    # Prepare the input data
    input_data = pd.DataFrame([[wine_type, grape_variety, gender, occasion]], columns=['Wine Type', 'Grape Variety', 'Gender', 'Occasion'])

    input_data['Wine Type'] = le_wine_type.transform(input_data['Wine Type'])
    input_data['Grape Variety'] = le_grape_variety.transform(input_data['Grape Variety'])
    input_data['Gender'] = le_gender.transform(input_data['Gender'])
    input_data['Occasion'] = le_occasion.transform(input_data['Occasion'])

    # Use the model to find the 3 nearest neighbors
    indices = knn.kneighbors(input_data, n_neighbors=3, return_distance=False)

    # Get the indices of the 3 nearest neighbors
    recommended_wines_indices = indices[0]

    # Retrieve full details of recommended wines
    recommended_wines = df.loc[recommended_wines_indices]

    # Inverse transform categorical columns
    recommended_wines['Wine Type'] = le_wine_type.inverse_transform(recommended_wines['Wine Type'])
    recommended_wines['Grape Variety'] = le_grape_variety.inverse_transform(recommended_wines['Grape Variety'])
    recommended_wines['Gender'] = le_gender.inverse_transform(recommended_wines['Gender'])
    recommended_wines['Occasion'] = le_occasion.inverse_transform(recommended_wines['Occasion'])
    recommended_wines['Title'] = le_title.inverse_transform(recommended_wines['Title'])

    # Select and return all columns
    return recommended_wines[['Title', 'Description', 'price', 'Country', 'IMG URL', 'Wine Type', 'Grape Variety', 'Gender', 'Occasion']]

def predict_trending_wine(recommended_wines):
    # Load the dataset containing wine data including sales volume
    wine_data_df = pd.read_csv('/content/TrendingWine.csv', encoding='ISO-8859-1')

    # Select relevant features and target variable
    features = ['Wine Type', 'Grape Variety', 'Gender', 'Occasion']  # Adjust as needed
    target = 'Sales Volume'

    # Convert categorical variables to numeric
    le_wine_type = LabelEncoder()
    le_grape_variety = LabelEncoder()
    le_gender = LabelEncoder()
    le_occasion = LabelEncoder()

    # Fit the encoders on the entire dataset for consistency
    wine_data_df['Wine Type'] = le_wine_type.fit_transform(wine_data_df['Wine Type'])
    wine_data_df['Grape Variety'] = le_grape_variety.fit_transform(wine_data_df['Grape Variety'])
    wine_data_df['Gender'] = le_gender.fit_transform(wine_data_df['Gender'])
    wine_data_df['Occasion'] = le_occasion.fit_transform(wine_data_df['Occasion'])

    # Transform the recommended wines DataFrame
    recommended_wines['Wine Type'] = le_wine_type.transform(recommended_wines['Wine Type'])
    recommended_wines['Grape Variety'] = le_grape_variety.transform(recommended_wines['Grape Variety'])
    recommended_wines['Gender'] = le_gender.transform(recommended_wines['Gender'])
    recommended_wines['Occasion'] = le_occasion.transform(recommended_wines['Occasion'])

    X = wine_data_df[features]
    y = wine_data_df[target]

    # Train a random forest regressor model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X, y)

    # Make predictions for the recommended wines
    predicted_sales_volumes = model.predict(recommended_wines[features])

    # Identify the trending wine based on predicted sales volumes
    trending_wine_index = predicted_sales_volumes.argmax()
    trending_wine = recommended_wines.iloc[trending_wine_index]

    return trending_wine['Title'], trending_wine['IMG URL']  # Return Title and Image URL only

# Example usage:
wine_type = 'White Wine'
grape_variety = 'Chardonnay'
gender = 'Female'
occasion = 'Summer Gatherings'

recommended_wines = recommend_wine(wine_type, grape_variety, gender, occasion)
print("Recommended Wines:")
print(recommended_wines[['Title', 'Description', 'price', 'Country', 'IMG URL']])

trending_wine_title, trending_wine_image_url = predict_trending_wine(recommended_wines)
print("\nTrending Wine:")
print("Title:", trending_wine_title)
print("Image URL:", trending_wine_image_url)


Recommended Wines:
                                               Title  \
28  Bread & Butter Winemaker's Selection Chardonnay    
26                     Morgan Bay Cellars Chardonnay   
20                       Ex Oppido Uitsig Chardonnay   

                                          Description     price       Country  \
28  Rich with creamy notes of vanilla bean, almond...  4,100.00           USA   
26  Appearance: Bright gold. Nose: Bright aromas o...  8,300.00           USA   
20  The wine features a complex nose that leads wi...  7,500.00  South Africa   

                                              IMG URL  
28  https://images.vivino.com/thumbs/tE1Nt99dSDO7C...  
26  https://images.vivino.com/thumbs/gPCeuravT9OIu...  
20  https://images.vivino.com/thumbs/Vh9tEAWmRLqth...  

Trending Wine:
Title: Ex Oppido Uitsig Chardonnay
Image URL: https://images.vivino.com/thumbs/Vh9tEAWmRLqth6kuCYz4Jw_pb_x600.png
